In [1]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
# from IPython import display
# from IPython.display import clear_output

from google.colab import drive
drive.mount('/content/drive/')
# path = "/content/drive/My Drive/HDSP/PnP-ADMM"
path = "/content/drive/My Drive/DATA"
os.chdir(path)

from os import listdir
from os.path import isfile, join

import numpy as np
import keras
import scipy.io
import pandas as pd
from scipy.io import loadmat

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

np.random.seed(1234)
#---------------------defined function used -------------------------------------------------------------------


# Defined protocol of input and ouput (it is recommended only read, not applied operator here, but it is possible)
def Input_image(image):
    images = loadmat(image).get('y')
    images = np.expand_dims(images,-1)
    # procesamiento
    return images/np.max(images)

def Oput_image(image):
    images = loadmat(image).get('y_m')
    images = np.expand_dims(images, -1)
    return images/np.max(images)

def load_sambles(data):
  data = data[['inimg']]
  inimg_name = list(data.iloc[:,0])
  samples = []
  for samp in inimg_name:
    samples.append(samp)
  return samples


class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, samples,PATH,batch_size=1,dim_input=(512, 512, 3), shuffle=True, dim_oput=(512, 512, 3)):
        'Initialization'
        self.dim_input = dim_input
        self.dim_oput = dim_oput
        self.batch_size = batch_size
        self.list_images = samples
        self.shuffle = shuffle
        self.PATH = PATH
        self.on_epoch_end()


    # falta __data_generation
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(len(self.list_images) / self.batch_size)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        images_name = [self.list_images[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(images_name)

        return X, y

    def on_epoch_end(self):
        'Update indexes after each epoch'
        self.indexes = np.arange(len(self.list_images))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, images_names):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim_input))  # Array de numpy con zeros de tamaño
        y = np.empty((self.batch_size, *self.dim_oput))
        # Generate data
        for i, file_name in enumerate(images_names):
            # Store sample
            X[i,] = Input_image(self.PATH + '/' + file_name)
            # Store class
            y[i,] = Oput_image(self.PATH + '/' + file_name)
        return X, y



def Build_data_set(IMG_WIDTH,IMG_HEIGHT,L_bands,L_imput,BATCH_SIZE,PATH,split_v):

  # Random split
  data_dir_list = os.listdir(PATH)
  N = len(data_dir_list)
  train_df = pd.DataFrame(columns=['inimg'])
  test_df = pd.DataFrame(columns=['inimg'])
  randurls = np.copy(data_dir_list)
  train_n = round(N * split_v)
  np.random.shuffle(randurls)
  tr_urls = randurls[:train_n]
  ts_urls = randurls[train_n:N]
  for i in tr_urls:
      train_df = train_df.append({'inimg': i}, ignore_index=True)
  for i in ts_urls:
      test_df = test_df.append({'inimg': i}, ignore_index=True)

  params = {'dim_input': (IMG_WIDTH, IMG_HEIGHT, L_imput),
            'dim_oput': (IMG_WIDTH, IMG_HEIGHT, L_bands),
            'batch_size': BATCH_SIZE,
            'PATH': PATH,
            'shuffle': True}

  partition_Train = load_sambles(test_df)
  partition_Test = load_sambles(train_df)

  train_generator = DataGenerator(partition_Train, **params)
  test_generator = DataGenerator(partition_Test, **params)

  train_dataset = tf.data.Dataset.from_generator(
      lambda: train_generator,
      (tf.float32, tf.float32))

  test_dataset = tf.data.Dataset.from_generator(
      lambda: test_generator,
      (tf.float32, tf.float32))

  return train_dataset,test_dataset

def PSNR_Metric(y_true, y_pred):
  return tf.reduce_mean(tf.image.psnr(y_true,y_pred,tf.reduce_max(y_true)))

# os.environ["CUDA_VISIBLE_DEVICES"]= '1'
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
# !pwd
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))


#----------------------------- directory of the spectral data set -------------------------                              # for windows
BATCH_SIZE = 1; L_bands    = 1; L_imput    = 1;
from scipy.io import loadmat
# from recoverynet import *     # Net build


#########################################################################################

ARAD = False # @ {type:"boolean"} ### IF TRUE MODIFIY NETWORK
if ARAD:
  IMG_WIDTH = 286; IMG_HEIGHT = 256;
  path_files  = os.getcwd()+'/ARAD/Y'
  nI = 10
else:
  IMG_WIDTH = 535; IMG_HEIGHT = 512;
  path_files  = os.getcwd()+'/REAL/Y'
  nI = 14
reTrain = False # @ {type:"boolean"}
gf=0.0  # @ {type:"raw"}
awgn=25 # @ {type:"raw"}
lr = 1e-4# @ {type:"raw"}
batch =  nI# @ {type:"raw"}

norm_inp="NO-F"  # @ {type:"raw"}
norm_out="NO-F"  # @ {type:"raw"}
loss =  'mean_squared_error'# @ {type:"string"}
model_path=os.getcwd()
net = "NO-NO"
old_cp_dir=model_path+'/Weights/Julian/' #@param {type:"raw"}
# if ARAD:
#     old_cp_dir += 'ARAD_'+str(gf)+'_'+str(awgn)+'_'+net+'/'
# else:
#     old_cp_dir += 'REAL/'




#########################################################################################

optimizad = tf.keras.optimizers.Adam(learning_rate=lr, amsgrad=False)

# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=new_cp_path,
#     save_weights_only=True,
#     verbose=1,
#     save_freq='epoch',
#     monitor='PSNR_Metric',
#     mode='max',
#     save_best_only=True)



Y_inp = None
for s in range(nI):
  if norm_inp!="" and ARAD:
    Temp = np.array(loadmat(path_files + '/y_ideal_'+str(s+1)+'_'+norm_inp)['y_ideal'])
  else:
    Temp = np.array(loadmat(path_files + '/y_ideal_'+str(s+1))['y_ideal'])
  if Y_inp is None:
    Y_inp = np.expand_dims(Temp, 0)
  else:
    Y_inp = np.concatenate((Y_inp, np.expand_dims(Temp, 0)),0)

Y_oup = None
for s in range(nI):
  if norm_out!="" and ARAD:
    Temp = np.array(loadmat(path_files + '/y_real_m_'+str(s+1)+'_'+str(gf)+'_'+str(awgn)+'_'+norm_out)['y_real_m'])
  elif ARAD:
    Temp = np.array(loadmat(path_files + '/y_real_m_'+str(s+1)+'_'+str(gf)+'_'+str(awgn))['y_real_m'])
  else:
    Temp = np.array(loadmat(path_files + '/y_real_'+str(s+1))['y_real'])
  if Y_oup is None:
    Y_oup = np.expand_dims(Temp, 0)
  else:
    Y_oup = np.concatenate((Y_oup, np.expand_dims(Temp, 0)),0)

Y_inp = np.expand_dims(Y_inp, -1)
Y_oup = np.expand_dims(Y_oup, -1)

Mounted at /content/drive/
Sun Sep 11 16:54:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                             

In [2]:
print(Y_inp.shape)
print(Y_oup.shape)

(14, 512, 535, 1)
(14, 512, 535, 1)


In [ ]:
def plot_results(records, tiempos, title, legend):
  import seaborn as sns
  sns.set_theme(style='darkgrid')
  plt.figure(figsize=(15, 5))

  plt.subplot(1, 2, 1)
  for rec in records.keys():
    plt.plot(records[rec].history['val_accuracy'], label='{}: {}'.format(legend, rec))

  plt.title(title)
  plt.legend()
  plt.ylabel('Accuracy')
  plt.xlabel('Epochs')

  plt.subplot(1, 2, 2)

  plt.bar(tiempos.keys(), tiempos.values())
  plt.xticks(rotation='vertical')
  plt.ylabel('Tiempo en segundos')
  plt.xlabel(legend)


In [ ]:
class Modelo:
  def __init__(self, input_size):
    self.input_dims = input_size
    self.input_layer = Input(input_size)
    self.model = self.input_layer
    self.convolucionales = []

  def build_unet(self, deepness=4, layers_per_level=2, filters=16, factor=1, kernel_size=3, dropout_ratio=0):

    filters_for_level = [filters * factor * f for f in range(1, deepness + 1)]

    # Going down
    # Level 0
    for _ in range(layers_per_level):
      self.build_conv2d(filters_for_level[0], kernel_size)
      
    self.build_dropout(dropout_ratio) # Drop out layer
    self.build_maxpool()  # Reduces dimentions

    # Level 1
    for _ in range(layers_per_level):
      self.build_conv2d(filters_for_level[1], kernel_size)
      
    self.build_dropout(dropout_ratio) # Drop out layer
    self.build_maxpool()  # Reduces dimentions

    # Level 2
    for _ in range(layers_per_level):
      self.build_conv2d(filters_for_level[2], kernel_size)
      
    self.build_dropout(dropout_ratio) # Drop out layer
    self.build_maxpool()  # Reduces dimentions

    # Level 3 - Embedding representation layer
    for _ in range(layers_per_level):
      self.build_conv2d(filters_for_level[3], kernel_size)
      
    self.build_dropout(dropout_ratio) # Drop out layer
    
    # Going Up
    # Level 2
    self.build_upsampling() # Increases dimentionality
    for _ in range(1, layers_per_level):
      self.build_conv2d(filters_for_level[2], kernel_size - 1) 
    self.build_conv2d_transpose(filters_for_level[2], 1, 2) # Some fancy layers
    self.build_concatenate(conv1=self.convolucionales[(layers_per_level * 3) - 1], conv2=self.convolucionales[-1]) # Connects to the other layers
    self.build_dropout(dropout_ratio)      

    # Level 1
    self.build_upsampling() # Increases dimentionality
    for _ in range(1, layers_per_level):
      self.build_conv2d(filters_for_level[1], kernel_size - 1) 
    self.build_conv2d_transpose(filters_for_level[1], 1, 2) # Some fancy layers
    self.build_concatenate(conv1=self.convolucionales[(layers_per_level * 2) - 1], conv2=self.convolucionales[-1]) # Connects to the other layers   
    self.build_dropout(dropout_ratio)      

    # Level 0
    self.build_upsampling() # Increases dimentionality
    for _ in range(1, layers_per_level):
      self.build_conv2d(filters_for_level[0], kernel_size - 1) 
    self.build_conv2d_transpose(filters_for_level[0], 1, 2) # Some fancy layers
    self.build_concatenate(conv1=self.convolucionales[(layers_per_level * 1) - 1], conv2=self.convolucionales[-1]) # Connects to the other layers   
    self.build_dropout(dropout_ratio) 
    
    # Final layer
    self.build_conv2d(1, 1)

  def get_model(self):
    return Model(self.input_layer, self.model)
    
  def build_conv2d(self, filters, kernel_size, activation='relu'):
    layer = Conv2D(filters, kernel_size, activation=activation, padding='same', kernel_initializer='he_normal')(self.model)
    self.convolucionales.append(layer)
    self.model = layer

  def build_maxpool(self, pooling_size=(2, 2)):
    self.model = MaxPooling2D(pool_size=pooling_size)(self.model)

  def build_upsampling(self, size=(2, 2)):
    self.model = UpSampling2D(size=size)(self.model)
  
  def build_concatenate(self, conv1, conv2, axis=3):
    self.model = concatenate([conv1, conv2], axis=axis)

  def build_conv2d_transpose(self, filters, dim1, dim2, activation='relu'):
    layer = Conv2DTranspose(filters, [dim1, dim2], activation=activation)(self.model)
    self.convolucionales.append(layer)
    self.model = layer

  def build_dropout(self, ratio):
    self.model = Dropout(ratio)(self.model)

model = Modelo(input_size=(512, 535, 1))
model.build_unet()
model.get_model().summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 535, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 535, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 512, 535, 16  2320        ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
import time
import pickle


def deep_comparison(optimizad, loss, metrics, epochs, verbose=0, input_dims=(256, 256, 12)):

  cant_conv_layers = [2, 4, 6, 8]
  folder_bin = "/content/drive/My Drive/DATA/julian"

  # Arreglos conteniendo los resultados del entrenamiento
  tiempos = {}
  records = {}

  for idx, cant_layers in enumerate(cant_conv_layers):

    print(idx + 1, "/3")
    print(input_dims)

    # Se crea el modelo 
    builder = Modelo(input_size=input_dims)
    builder.build_unet(layers_per_level=cant_layers)
    model = builder.get_model()
    model.compile(optimizer=optimizad, loss=loss, metrics=metrics)

    model.compile

    # Se mide el tiempo para el momento antes de entrenar
    start = time.time()

    # Se entrena al modelo con los datos proporcionados
    records[str(idx)] = model.fit(Y_inp, Y_oup, 
                        epochs=epochs, 
                        verbose=verbose)

    # Se mide el tiempo para el momento despues de entrenar
    end = time.time()

    # Se registra el tiempo de ejecución 
    tiempos[str(idx)]=end-start

    # Se guarda el modelo en el drive
    model.save(old_cp_dir + 'variacion_profundidad_{}.h5'.format(idx))

  # Se guardan los resultados de entrenamiento en el drive
  with open('/'.join([old_cp_dir, 'resultados_deep_var.pkl']), 'wb') as file:
    resultados = {
        'records': records,
        'tiempos': tiempos
    }
    pickle.dump(resultados, file)

  return records, tiempos

records, tiempos = deep_comparison(optimizad=optimizad, loss=loss, metrics = [PSNR_Metric], epochs=1000, input_dims=(IMG_HEIGHT, IMG_WIDTH, L_bands))

1 /3
(512, 535, 1)
2 /3
(512, 535, 1)
3 /3
(512, 535, 1)
4 /3
(512, 535, 1)


## Se muestran los resultados

In [ ]:
plot_results(records, tiempos, "Comparacion Profundidades", ""):

In [ ]:
import time
import pickle


def dropout_comparison(optimizad, loss, metrics, epochs, verbose=0, input_dims=(256, 256, 12)):

  porcentaje_dropout = [0.05, 0.1, 0.2, 0.4, 0.6, 0.9]
  folder_bin = "/content/drive/My Drive/DATA/julian"

  # Arreglos conteniendo los resultados del entrenamiento
  tiempos = {}
  records = {}

  for idx, drop_out_per in enumerate(porcentaje_dropout):

    cant_layers=2
    print("Corriendo: {}/{}".format(idx + 1, len(porcentaje_dropout)))
    print(input_dims)

    # Se crea el modelo 
    builder = Modelo(input_size=input_dims)
    builder.build_unet(dropout_ratio=drop_out_per)
    model = builder.get_model()
    model.compile(optimizer=optimizad, loss=loss, metrics=metrics)

    model.compile

    # Se mide el tiempo para el momento antes de entrenar
    start = time.time()

    # Se entrena al modelo con los datos proporcionados
    records[str(idx)] = model.fit(Y_inp, Y_oup, 
                        epochs=epochs, 
                        verbose=verbose)

    # Se mide el tiempo para el momento despues de entrenar
    end = time.time()

    # Se registra el tiempo de ejecución 
    tiempos[str(idx)]=end-start

    # Se guarda el modelo en el drive
    model.save(old_cp_dir + 'variacion_dropout_{}.h5'.format(idx))

  # Se guardan los resultados de entrenamiento en el drive
  with open('/'.join([old_cp_dir, 'resultados_dropout_var.pkl']), 'wb') as file:
    resultados = {
        'records': records,
        'tiempos': tiempos
    }
    pickle.dump(resultados, file)

  return records, tiempos

records, tiempos = dropout_comparison(optimizad=optimizad, loss=loss, metrics = [PSNR_Metric], epochs=1000, input_dims=(IMG_HEIGHT, IMG_WIDTH, L_bands))

Corriendo: 1/6
(512, 535, 1)


In [ ]:

epochs =  10# @param {type:"number"}

#-------------Net_model----------------------------------------------------------------
model = UNetL2(input_size=(IMG_HEIGHT,IMG_WIDTH,L_bands))
model.compile(optimizer=optimizad, loss=loss,metrics = [PSNR_Metric])
history = model.fit(x=Y_inp,y=Y_oup, epochs=epochs)#, callbacks=[model_checkpoint_callback])
last_psnr=history.history['PSNR_Metric'][-1]
best_psnr=np.max(history.history['PSNR_Metric'])
iter_psnr=history.history['PSNR_Metric'].index(best_psnr)
best_loss=np.min(history.history['loss'])
iter_loss=history.history['loss'].index(best_loss)
name="Dropout_0.5" # @param {type:"string"}
name+=', '+str(best_psnr)+', '+str(best_loss)+', '+str(epochs)+'.h5'
print(name)
model.save(old_cp_dir+name)

#.------------ seee the accuracy---------------
plt.plot(history.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show()
plt.plot(history.history['PSNR_Metric'], label='PSNR_Metric')
plt.xlabel('Epoch')
plt.ylabel('PSNR_Metric')
plt.show()

Epoch 1/10
1/1 [==============================] - 20s 20s/step - loss: 0.0063 - PSNR_Metric: 22.5201
Epoch 2/10
1/1 [==============================] - 1s 635ms/step - loss: 0.0044 - PSNR_Metric: 24.1216
Epoch 3/10
1/1 [==============================] - 1s 641ms/step - loss: 0.0035 - PSNR_Metric: 25.0543
Epoch 4/10
1/1 [==============================] - 1s 641ms/step - loss: 0.0031 - PSNR_Metric: 25.4832
Epoch 5/10
1/1 [==============================] - 1s 639ms/step - loss: 0.0030 - PSNR_Metric: 25.6522
Epoch 6/10
1/1 [==============================] - 1s 635ms/step - loss: 0.0030 - PSNR_Metric: 25.6372
Epoch 7/10
1/1 [==============================] - 1s 641ms/step - loss: 0.0030 - PSNR_Metric: 25.5792
Epoch 8/10


KeyboardInterrupt: ignored

In [4]:
output_dim = 300#@param {type:"raw"}
embeddings_initializer='uniform'#@param {type:"raw"}
embeddings_regularizer=None#@param {type:"raw"}
activity_regularizer= None#@param {type:"raw"}
embeddings_constraint= None#@param {type:"raw"}
mask_zero= False#@param {type:"raw"}

In [ ]:
units = 128#@param {type:"raw"}
activation="tanh" #@param {type:"raw"}
use_bias=True #@param {type:"raw"}
kernel_initializer="glorot_uniform"  #@param {type:"raw"}
recurrent_initializer="orthogonal" #@param {type:"raw"}
bias_initializer="zeros" #@param {type:"raw"}
kernel_regularizer=None #@param {type:"raw"}
recurrent_regularizer=None #@param {type:"raw"}
bias_regularizer=None #@param {type:"raw"}
activity_regularizer=None #@param {type:"raw"}
kernel_constraint=None #@param {type:"raw"}
recurrent_constraint=None #@param {type:"raw"}
bias_constraint=None #@param {type:"raw"}
dropout=0.0 #@param {type:"raw"}
recurrent_dropout=0.0 #@param {type:"raw"}
return_sequences=True #@param {type:"raw"}
return_state=False #@param {type:"raw"}
go_backwards=False #@param {type:"raw"}
stateful=False #@param {type:"raw"}
unroll=False #@param {type:"raw"}

In [5]:
def create_model(input_dim, input_length):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
      units,
      activation=activation,
      use_bias=use_bias,
      kernel_initializer=kernel_initializer,
      recurrent_initializer=recurrent_initializer,
      bias_initializer=bias_initializer,
      kernel_regularizer=kernel_regularizer,
      recurrent_regularizer=recurrent_regularizer,
      bias_regularizer=bias_regularizer,
      activity_regularizer=activity_regularizer,
      kernel_constraint=kernel_constraint,
      recurrent_constraint=recurrent_constraint,
      bias_constraint=bias_constraint,
      dropout=dropout,
      recurrent_dropout=recurrent_dropout,
      return_sequences=True,
      return_state=return_state,
      go_backwards=go_backwards,
      stateful=stateful,
      unroll=unroll)))

  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
      units,
      activation=activation,
      use_bias=use_bias,
      kernel_initializer=kernel_initializer,
      recurrent_initializer=recurrent_initializer,
      bias_initializer=bias_initializer,
      kernel_regularizer=kernel_regularizer,
      recurrent_regularizer=recurrent_regularizer,
      bias_regularizer=bias_regularizer,
      activity_regularizer=activity_regularizer,
      kernel_constraint=kernel_constraint,
      recurrent_constraint=recurrent_constraint,
      bias_constraint=bias_constraint,
      dropout=dropout,
      recurrent_dropout=recurrent_dropout,
      return_sequences=True,
      return_state=return_state,
      go_backwards=go_backwards,
      stateful=stateful,
      unroll=unroll)))

  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
      units,
      activation=activation,
      use_bias=use_bias,
      kernel_initializer=kernel_initializer,
      recurrent_initializer=recurrent_initializer,
      bias_initializer=bias_initializer,
      kernel_regularizer=kernel_regularizer,
      recurrent_regularizer=recurrent_regularizer,
      bias_regularizer=bias_regularizer,
      activity_regularizer=activity_regularizer,
      kernel_constraint=kernel_constraint,
      recurrent_constraint=recurrent_constraint,
      bias_constraint=bias_constraint,
      dropout=dropout,
      recurrent_dropout=recurrent_dropout,
      return_sequences=False,
      return_state=return_state,
      go_backwards=go_backwards,
      stateful=stateful,
      unroll=unroll)))

  model.add(tf.keras.layers.Dense(units=128, 
                                    activation='relu',
                                    name='dense1') )
  model.add(tf.keras.layers.Dense(units = 1,
                                  activation='softmax',
                                  name='ouput_layer'))
  model.summary()
  return model


In [7]:
def create_simple_model(input_dim, input_length):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(
      units,
      activation=activation,
      use_bias=use_bias,
      kernel_initializer=kernel_initializer,
      recurrent_initializer=recurrent_initializer,
      bias_initializer=bias_initializer,
      kernel_regularizer=kernel_regularizer,
      recurrent_regularizer=recurrent_regularizer,
      bias_regularizer=bias_regularizer,
      activity_regularizer=activity_regularizer,
      kernel_constraint=kernel_constraint,
      recurrent_constraint=recurrent_constraint,
      bias_constraint=bias_constraint,
      dropout=dropout,
      recurrent_dropout=recurrent_dropout,
      return_sequences=True,
      return_state=return_state,
      go_backwards=go_backwards,
      stateful=stateful,
      unroll=unroll)))

  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(
      units,
      activation=activation,
      use_bias=use_bias,
      kernel_initializer=kernel_initializer,
      recurrent_initializer=recurrent_initializer,
      bias_initializer=bias_initializer,
      kernel_regularizer=kernel_regularizer,
      recurrent_regularizer=recurrent_regularizer,
      bias_regularizer=bias_regularizer,
      activity_regularizer=activity_regularizer,
      kernel_constraint=kernel_constraint,
      recurrent_constraint=recurrent_constraint,
      bias_constraint=bias_constraint,
      dropout=dropout,
      recurrent_dropout=recurrent_dropout,
      return_sequences=True,
      return_state=return_state,
      go_backwards=go_backwards,
      stateful=stateful,
      unroll=unroll)))

  model.add(tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(
      units,
      activation=activation,
      use_bias=use_bias,
      kernel_initializer=kernel_initializer,
      recurrent_initializer=recurrent_initializer,
      bias_initializer=bias_initializer,
      kernel_regularizer=kernel_regularizer,
      recurrent_regularizer=recurrent_regularizer,
      bias_regularizer=bias_regularizer,
      activity_regularizer=activity_regularizer,
      kernel_constraint=kernel_constraint,
      recurrent_constraint=recurrent_constraint,
      bias_constraint=bias_constraint,
      dropout=dropout,
      recurrent_dropout=recurrent_dropout,
      return_sequences=False,
      return_state=return_state,
      go_backwards=go_backwards,
      stateful=stateful,
      unroll=unroll)))

  model.add(tf.keras.layers.Dense(units=128, 
                                    activation='relu',
                                    name='dense1') )
  model.add(tf.keras.layers.Dense(units = 1,
                                  activation='softmax',
                                  name='ouput_layer'))
  model.summary()
  return model


In [6]:
loss = 'categorical_crossentropy' #@param {type:"raw"}
metrics = ['accuracy'] #@param {type:"raw"}
optimizer = 'adam' #@param {type:"raw"}
epochs = 5 #@param {type:"raw"}
verbose = 1 #@param {type:"raw"}

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

callbacks = [model_checkpoint_callback]

In [25]:
# Process the data

def process_images(k_splits, img_arr):
  n_images, length, width, channels = img_arr.shape
  img_length = width * length

  vectorized = img_arr.flatten()
  v_imgs = [vectorized[idx:idx + img_length] for idx in range(0, vectorized.shape[0], img_length)]

  
  

process_images(2, Y_inp)

535
14
